In [5]:
!pip install -U --upgrade --quiet langchain-google-vertexai langchain-google-genai langchain-community langchain unstructured lark chromadb

Installing all necessary python modules

In [ ]:
!pip show langchain langchain-core

Name: langchain
Version: 0.1.12
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
---
Name: langchain-core
Version: 0.1.32
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, jsonpatch, langsmith, packaging, pydantic, PyYAML, requests, tenacity
Required-by: langchain, langchain-community, langchain-google-genai, langchain-google-vertexai, langchain-text-splitters


In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI


from IPython.display import display
from IPython.display import Markdown

os.environ["GOOGLE_API_KEY"] = userdata.get('KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

Setting up the environment, connecting my API key to be able to access the Gemini through the Colab environment

Want to get a list of models that I can use....

In [6]:
models = [m for m in genai.list_models()]
models

[Model(name='models/chat-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 Chat (Legacy)',
       description='A legacy text-only model optimized for chat conversations',
       input_token_limit=4096,
       output_token_limit=1024,
       supported_generation_methods=['generateMessage', 'countMessageTokens'],
       temperature=0.25,
       top_p=0.95,
       top_k=40),
 Model(name='models/text-bison-001',
       base_model_id='',
       version='001',
       display_name='PaLM 2 (Legacy)',
       description='A legacy model that understands text and generates text as an output',
       input_token_limit=8196,
       output_token_limit=1024,
       supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
       temperature=0.7,
       top_p=0.95,
       top_k=40),
 Model(name='models/embedding-gecko-001',
       base_model_id='',
       version='001',
       display_name='Embedding Gecko',
       description='Obtai

Testing that the API keys are in fact working and that I can access/test Gemini from within this environment

In [8]:
model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7)

result = model.invoke("Waht is an LLM?")

Markdown(result.content)

Large Language Model (LLM)

Can begin actually working on the chaining

In [9]:
from langchain_core.messages import HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Define the initial prompt template for generating project architecture
architecture_prompt_template = """Given the project description below, generate an initial architecture outline suitable for GCP using Terraform. Keep it concise and to the point.

Project Description:
{project_description}

User Architecture Feedback:
{user_architecture_feedback}

Initial Architecture Outline:"""

# Define the prompt template for generating Terraform code based on confirmed architecture
terraform_prompt_template = """
Given the confirmed architecture outline below, generate the full Terraform code necessary to deploy the project on GCP. If you are asked for a revision, please make minimal changes to the existing Terraform code to accommodate the new request. Do not generate completely new code based on the asked revision. Try to find the most optimal way to add the new request while still maintaining the existing code structure.

Confirmed Architecture Outline:
{confirmed_architecture}

Existing Terraform Code:
{existing_terraform_code}

User Feedback:
{user_terraform_feedback}

Revised Terraform Code:"""

# Custom prompt templates
architecture_prompt = PromptTemplate.from_template(architecture_prompt_template)
terraform_code_prompt = PromptTemplate.from_template(terraform_prompt_template)

# Define the interaction chain for architecture generation and confirmation
architecture_chain = (
    {"project_description": RunnablePassthrough(), "user_architecture_feedback": RunnablePassthrough()}
    | architecture_prompt
    | model
    | StrOutputParser()
)

# Define the interaction chain for Terraform code generation
terraform_code_chain = (
    {"confirmed_architecture": RunnablePassthrough(), "existing_terraform_code": RunnablePassthrough(), "user_terraform_feedback": RunnablePassthrough()}
    | terraform_code_prompt
    | model
    | StrOutputParser()
)

# Define the prompt template for suggesting improvements
improvement_prompt_template = """
Given the confirmed architecture outline below, suggest potential improvements that could enhance the project's efficiency, scalability, or reliability.

Confirmed Architecture Outline:
{confirmed_architecture}

Suggested Improvements:"""

improvement_prompt = PromptTemplate.from_template(improvement_prompt_template)

# Define the interaction chain for improvement suggestions
improvement_chain = (
    {"confirmed_architecture": RunnablePassthrough()}
    | improvement_prompt
    | model
    | StrOutputParser()
)

# Get the project description from the user
project_description = input("Enter the project description: ")

user_architecture_feedback = "" # Initializing user feedback
user_terraform_feedback = "" # Initializing user feedback
existing_terraform_code = "" # Initializing existing Terraform code

while True:
    # Run the architecture_chain
    architecture = architecture_chain.invoke({"project_description": project_description, "user_architecture_feedback": user_architecture_feedback})

    # Display the generated architecture to the user
    print("Generated Architecture: ", architecture)

    # Ask for user confirmation
    user_confirmation = input("User confirms this architecture? (yes/no): ")

    # If user confirms, break the loop
    if user_confirmation.lower() == 'yes':
        break
    else:
        user_architecture_feedback = input("Please describe the issues you have with the architecture: ")

# After user confirmation, run the improvement_chain
improvements = improvement_chain.invoke({"confirmed_architecture": architecture})

# Display the suggested improvements to the user
print("Suggested Improvements: ", improvements)

# Ask for user confirmation
user_confirmation = input("Would you like to incorporate these improvements? (yes/no): ")

# If user confirms, update the architecture
if user_confirmation.lower() == 'yes':
    architecture += "\n\nIncorporated Improvements:\n" + improvements

# After user confirmation, run the terraform_code_chain
while True:
    # Invoke the Terraform code generation chain
    new_terraform_code = terraform_code_chain.invoke({"confirmed_architecture": architecture, "existing_terraform_code": existing_terraform_code, "user_terraform_feedback": user_terraform_feedback})

    # Display the generated Terraform code
    print("Generated Terraform Code: ", new_terraform_code)

    # Ask for user confirmation
    user_confirmation = input("User confirms this Terraform code? (yes/no): ")

    # If user confirms, break the loop
    if user_confirmation.lower() == 'yes':
        existing_terraform_code = new_terraform_code  # Update existing Terraform code
        break
    else:
        user_previous_feedback = user_terraform_feedback  # Store previous user feedback
        user_terraform_feedback = input("Please describe the issues you have with the Terraform code: ")

        # Incorporate both previous and new feedback into the architecture
        architecture += "\n\nUser Feedback from Previous Iteration:\n" + user_previous_feedback + "\n"
        architecture += "\n\nUser Feedback from Current Iteration:\n" + user_terraform_feedback + "\n"

# Save the final Terraform code to a .hcl file
with open('terraform_code.hcl', 'w') as file:
    file.write(existing_terraform_code)


Enter the project description: I want you to create me a website
Generated Architecture:  **Infrastructure:**

* **Compute Engine:** Host web application and database
* **Cloud Storage:** Store website content, backups, and logs
* **Cloud Networking:** Provide network connectivity and security
* **Load Balancer:** Distribute traffic across web instances

**Services:**

* **Cloud SQL:** Managed MySQL database
* **Cloud DNS:** Manage DNS records for website
* **Cloud CDN:** Improve website performance by caching static content
* **Cloud Monitoring:** Monitor website and infrastructure health

**Tools:**

* **Terraform:** Manage and provision cloud resources
* **CI/CD Pipeline:** Automate infrastructure deployment and updates
User confirms this architecture? (yes/no): yes
Suggested Improvements:  **Efficiency:**

* **Consider using autoscaling:** Automatically adjust the number of Compute Engine instances based on traffic load to optimize resource utilization and cost.
* **Implement cachi